In [1]:
!nvidia-smi

Wed Sep  6 04:33:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone -q https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q

In [12]:
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


DataSet

In [15]:
!wget -q https://github.com/Beomi/KoAlpaca/raw/main/ko_alpaca_data.json

In [17]:

import json

with open('ko_alpaca_data.json','r') as f:
    dataset = json.load(f)

dataset[:5]

[{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'input': '',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '세 가지 기본 색은 무엇인가요?',
  'input': '',
  'output': '기본 색은 빨강, 파랑, 노랑입니다.'},
 {'instruction': '원자의 구조를 설명하세요.',
  'input': '',
  'output': '원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있고 전자는 주변에 있습니다.'},
 {'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
  'input': '',
  'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'},
 {'instruction': '어려운 결정을 내려야 했던 때를 설명하세요.',
  'input': '',
  'output': '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.'}]

FineTune

In [18]:
import json

prompt_template = {

       "description": "Alpaca-LoRA Custom 템플릿",
       "prompt_input": (
          "Below is an instruction that describes a task, paired with an input that provides further context. \n"
          "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다. \n\n"
          "Write a response that appropriately completes the request. \n요청을 적절히 완료하는 응답을 작성하세요. \n\n"
          "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response: \n"
       ),
       "prompt_no_input": (
          "Below is an instruction that describes a task. \n"
          "아래는 작업을 설명하는 명령어입니다. \n\n"
          "Write a response that appropriately completes the request. \n 명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요. \n\n"
          "### Instruction(명령어):\n{instruction}\n\n### Response: \n"
       ),
       "response_split": "### Response:",
}
with open('templates/custom.json','w',encoding ='utf-8') as f:
  json.dump( prompt_template , f,ensure_ascii=False)

In [ ]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'ko_alpaca_data.json' \
    --output_dir './output' \
    --num_epochs 1 \
    --learning_rate 3e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

2023-09-06 04:53:40.507724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Training Alpaca-LoRA model with params:
base_model: decapoda-research/llama-7b-hf
data_path: ko_alpaca_data.json
output_dir: ./output
batch_size: 512
micro_batch_size: 16
num_epochs: 1
learning_rate: 0.0003
cutoff_len: 256
val_set_size: 2000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: False
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: custom

Loading checkpoint shards: 100% 33/33 [01:13<00:00,  2.23s/it]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are us

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/LLaMA-Alpaca-LoRA
!cp -a output /content/drive/MyDrive/LLaMa-Alpaca-LoRA

In [ ]:
!python generate.py \
  --base_model 'decapoda-research/llama-7b-hf' \
  --lora_weights '/content/drive/MyDrive/LLaMa-Alpaca-LoRA/output' \
  --prompt_template 'custom' \
  --share_gradio

Test on Gradio